In [10]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import concurrent.futures
import time
import re
from collections import OrderedDict
import pandas as pd
import numpy as np
from datetime import datetime
import sys
import urllib.parse


# VARIABLES
all_genres = ['action','adventure','animation','comedy','crime','documentary','drama','family','fantasy','history','horror','music','mystery','romance','science fiction','thriller','tv movie','war','western']
top_genres = {
    'action': 0,
    'adventure': 0,
    'animation': 0,
    'comedy': 0,
    'crime': 0,
    'documentary': 0,
    'drama': 0,
    'family': 0,
    'fantasy': 0,
    'history': 0,
    'horror': 0,
    'music': 0,
    'mystery': 0,
    'romance': 0,
    'science fiction': 0,
    'thriller': 0,
    'tv movie': 0,
    'war': 0,
    'western': 0
}
list_urls = []
titles = []
ratings_list = []
release_years = []
data = {}

# GETTING THE URLS
## First Page
base_url = 'https://letterboxd.com'
user_name = input('Enter user name: ')
r = requests.get(f'{base_url}/{user_name}/films/diary/')
soup = BeautifulSoup(r.content, "html.parser")
list_urls = [base_url + a.get('href') for a in soup.select('h3>a[href]')]
if list_urls == []:
    print('No films found')
    sys.exit(1)
## Subsequent Pages
links = soup.find_all('li', class_="paginate-page")
try:
    start_character = str(links[-1]).find('/page/') + 6
    end_character = str(links[-1]).find('/">')
    end_page = int(str(links[-1])[start_character:end_character])
except IndexError:
    end_page = 1
if end_page >1:
    for page in range(2, end_page + 1):
        r = requests.get(f'{base_url}/{user_name}/films/diary/page/{page}')
        soup = BeautifulSoup(r.content, "html.parser")
        new_urls = [base_url + a.get('href') for a in soup.select('h3>a[href]')]
        list_urls.extend(new_urls)

# GETTING THE TITLES, RELEASE YEAR, RATINGS, AND GENRES + DATA FORMATTING
for url in list_urls:
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    ## Getting the Titles and Release Year
    span = soup.find_all('span', class_="film-title-wrapper")
    span_soup = BeautifulSoup('\n'.join(map(str, span)))
    a_list = list(span_soup.find_all('a'))
    count = 0
    for a in a_list:
        a_str =str(a)
        start_chr = a_str.find('">')+2
        end_chr = a_str.find("</")
        count += 1
        if count == 1:
            titles.append(a_str[start_chr:end_chr])
        if count == 2:
            release_years.append(a_str[start_chr:end_chr])
        if count == 3:
            print('uh oh, Error at titles/year')
            break
    ## Getting the Ratings
    script_str = str(soup.find_all('script', type="application/ld+json"))
    start_chr = script_str.find('"ratingValue":')+14
    end_chr = script_str.find(',"worstRating"')
    if end_chr == -1:
        ratings_list.append('null')
        continue
    ratings_list.append(script_str[start_chr:end_chr])
    ## Getting the Genres
    all_script_tags = soup.find_all("script")
    script_list = []
    movie_genres = []
    for i in all_script_tags:
        script_list.append(str(i))
    raw_script_tag = script_list[-1]
    for i in all_genres:
        if i in raw_script_tag:
            movie_genres.append(i)
            top_genres[i] += 1
    ## Formatting the Data Dictionary
    if titles[-1] not in data:
        data[titles[-1]] = [release_years[-1],ratings_list[-1], movie_genres]

# SORTING GENRE VALUES
sorted_top_genres = sorted(top_genres.items(), key=lambda item: item[1], reverse=True)

# =====================================================================================================

top_3 = [genre for genre, count in sorted_top_genres[:3]]

soup = BeautifulSoup('', 'html.parser')

# use webdriver to open letterboxd based on rating and genre
driver = webdriver.Chrome()

# get soup of all genres combined
driver.get(f"https://letterboxd.com/films/genre/{'+'.join(top_3)}/")
time.sleep(1)
html_doc = driver.page_source
soup1 = BeautifulSoup(html_doc, 'html.parser')
soup.extend(soup1.contents)
time.sleep(0.5)
# get soup of each genre separately
for i in [0,1,2]:
    for j in ['NA', 'page/2/']: # page2 for more data
        soup2 = j
        if j == 'NA':
            driver.get(f"https://letterboxd.com/films/genre/{top_3[i]}/")
        else:
            driver.get(f"https://letterboxd.com/films/genre/{top_3[i]}/{j}")
        time.sleep(1)
        html_doc = driver.page_source
        soup2 = BeautifulSoup(html_doc, 'html.parser')
        soup.extend(soup2.contents)
        time.sleep(0.5)
# get soup of old movies
for i in [0,1,2]:
    for j in ['9','8','7']: # old movies
        soup2 = j
        driver.get(f"https://letterboxd.com/films/popular/decade/19{j}0s/genre/{top_3[i]}/")
        time.sleep(1)
        html_doc = driver.page_source
        soup2 = BeautifulSoup(html_doc, 'html.parser')
        soup.extend(soup2.contents)
        time.sleep(0.5)

driver.quit()

# parse out the tags that have the film info
initial_poster_lists = soup.find_all('li', class_='listitem poster-container')

# removes duplicate entries
poster_lists = list(OrderedDict.fromkeys(initial_poster_lists))

# get the url
url_list = []
for poster in poster_lists:
    link_tag = poster.find('div')
    link_url = link_tag['data-poster-url'].replace('image-150/','')
    url_list.append(f'https://letterboxd.com{link_url}')

# get the title, date, and rating string
title_date_ratingS = []
for poster in poster_lists:
    info_tag = poster.find('a')
    title_date_rating = info_tag['data-original-title']
    title_date_ratingS.append(title_date_rating)

# get the title and rating, then put in a dictionary
title_rating_dict = {}
pattern = r"([^\(]+) \((\d{4})\) ([\d.]+)"
for info in title_date_ratingS:
    match = re.search(pattern, info)
    title = match.group(1)
    release = match.group(2)
    rating = match.group(3)
    if title in title_rating_dict.keys():
        title += '1'
        title_rating_dict[title] = [release, rating]
    else:
        title_rating_dict[title] = [release, rating]

# function to get genre and theme
def scrape_url(url):
    response = requests.get(url).content
    soup = BeautifulSoup(response, 'html.parser')
    # Extract the desired data from the soup object
    genre_theme_tags = soup.find_all('a', {'href': re.compile(r'/films/genre/(.+)')}) # if want themes: (genre|theme)
    listt = []
    for tag in genre_theme_tags:
        genre_theme = tag.text
        listt.append(genre_theme)
    return listt

# Use concurrent.futures to make all the requests in parallel
with concurrent.futures.ThreadPoolExecutor() as executor:
    genre_theme_lists = list(executor.map(scrape_url, url_list))

for i in range(len(title_rating_dict)):
    title_rating_dict[list(title_rating_dict.keys())[i]] += [genre_theme_lists[i]]

# convert title_rating_dict to dataframe
search_results_df = pd.DataFrame(title_rating_dict).T
search_results_df = search_results_df.rename(columns={0: 'Release Year', 1: 'Rating', 2: 'Genres'})
search_results_df['movie_url'] = url_list
sorted_search_df = search_results_df.sort_values(by=['Rating'], ascending=False)
sorted_search_df.reset_index(inplace=True)
sorted_search_df = sorted_search_df.rename(columns={'index':'movie title'})

sorted_search_df['Rating'] = sorted_search_df['Rating'].astype(float)
sorted_search_df['Release Year'] = sorted_search_df['Release Year'].astype(int)

#  ====================================================

# remove watched
search_results_df1=sorted_search_df.copy()
diary=list(data.keys())
for entry in diary:
    if entry in list(search_results_df1["movie title"]):
        search_results_df1=search_results_df1.drop(search_results_df1[search_results_df1["movie title"]==entry].index)

def get_rotten_tomatoes_url(movie_name, year):
    year=str(year)
    query = urllib.parse.quote(movie_name)
    url = f"https://www.rottentomatoes.com/search?search={query}"
    response = requests.get(url)
    movie=response.text
    soup = BeautifulSoup(movie, 'html.parser')
    URLS=soup.find_all('a',class_='unset')
    a=[]
    links=[]
    dates=[]
    titles=[]
    x=0
    y=0
    parse_title=["/n"," "]
    for tag in URLS:
        if "thumbnail" in str(tag):
            a.append(tag)
    for tag in a:
        links.append(str(tag)[str(tag).find("https"):(str(tag).find(" slot")-1)])
    datetag=soup.find_all('search-page-media-row')
    for tag in datetag:
        dates.append(str(tag)[str(tag).find("releaseyear")+13:str(tag).find("releaseyear")+17])
    while x!=len(datetag):
        lee=datetag[x].text.replace("\n","")
        lee=lee.replace("  ","")
        titles.append(lee)
        x+=1
    while y!=len(datetag):
        if dates[y]==year and titles[y]==movie_name:
            return links[y]
        y+=1

def get_movie_reviews(movie_title):
    unnecessary_punctuation_second = r"$@[].,'#()-\"!?’_;:/…"
    y=0
    clean_movie=movie_title
    while y!=len(unnecessary_punctuation_second):
        clean_movie=clean_movie.replace(unnecessary_punctuation_second[y],"")
        y+=1
    movie_string=clean_movie.split()
    x=0
    Movie=""
    while x!=len(movie_string):
        Movie+=movie_string[x]
        if x==len(movie_string)-1:
            ""
        else:
            Movie+="_"
        x+=1
    z=0
    url = f"https://www.rottentomatoes.com/m/{Movie}/reviews"
    response = requests.get(url)
    reviews=response.text
    soup = BeautifulSoup(reviews, 'html.parser')
    presented_review=soup.find_all('p',class_='review-text')[z].text
    review_critic=soup.find_all('a',class_='display-name')[z].text
    score=soup.find_all('score-icon-critics')
    tag=0
    number=len(score)
    score1=score.copy()
    while tag!=number:
        if "4" not in str(score1[tag]):
            score.remove(score1[tag])
        else:
            break
        tag+=1
    score
    score2=score[z]
    senti=str(score2)
    parse_critics = ["\n","  "]
    punct=0
    space=0
    while "NEGATIVE" in senti:
        z+=1
        score2=score[z]
        senti=str(score2)
        presented_review=soup.find_all('p',class_='review-text')[z].text
        review_critic=soup.find_all('a',class_='display-name')[z].text
        
    while punct!=len(parse_critics):
            review_critic=review_critic.replace(parse_critics[punct],"")
            punct+=1
    while space!=len(parse_critics):
            presented_review=presented_review.replace(parse_critics[space]," ")
            space+=1
    return presented_review+" ("+review_critic+", Rotten Tomatoes)"
    
    
def movie_reception(function, movie_title, year): #MAIN FUNCTION
    
    #--------------------------------------------
    year=str(year)
    try:
        movie_title1=movie_title 
        unnecessary_punctuation_first = r"$@[].,'#()-\"!?’_;:/…"
        y=0
        while y!=len(unnecessary_punctuation_first):
            movie_title1=movie_title1.replace(unnecessary_punctuation_first[y]," ")
            y+=1   
        movie_title1=movie_title1.replace("&",'and')
        movie_copy=movie_title1
        movie_title1=movie_title1+" "+year
        try:
            return function(movie_title1)
        except Exception:      
            return function(movie_copy)
    except Exception:
        movie_title2=movie_title
        movie_title2=movie_title2.replace("&",'and')
        movie_copy=movie_title2
        movie_title2=movie_title2+" "+year
        try:
            return function(movie_title2)
        except Exception:
            try:
                return function(movie_copy)
            except Exception:
                if function==get_movie_reviews:
                    return "No reviews found on Rotten Tomatoes"
                else:
                    return "not a valid function"

# convert diary dictionary to dataframe
diary_df = pd.DataFrame(data).T
diary_df = diary_df.rename(columns={0: 'Release Year', 1: 'Rating', 2: 'Genres'})
diary_df['Release Year'] = diary_df['Release Year'].astype(int)
current_year = datetime.now().year
edited_year = pd.cut(diary_df['Release Year'], 
              bins=[1899, current_year - 21, current_year - 6, current_year],
              labels=['old', 'notsorecent', 'recent'])
diary_df['Release Year'] = edited_year

# Function to recommend 15 random movies
def recommend_movies(df):
    # Filter the DataFrame by Year
    recent_movies = df[df['Release Year'] >= current_year - 5]
    notsorecent_movies = df[(df['Release Year'] < current_year - 5) & (df['Release Year'] >= current_year - 20)]
    old_movies = df[df['Release Year'] < current_year - 20]
    
    # Filter the DataFrame to only include movies with a rating of 3.5 or higher
    high_rated_movies1 = recent_movies[recent_movies['Rating'] >= 3.5]
    high_rated_movies2 = notsorecent_movies[notsorecent_movies['Rating'] >= 3.5]
    high_rated_movies3 = old_movies[old_movies['Rating'] >= 3.5]
    
    grouped = diary_df.groupby('Release Year')

    # calculate percentage of movies from the 3 'periods'
    recent_num = round((int(len(grouped.get_group('recent'))) / len(diary_df)) * 15)
    notsorecent_num = round((int(len(grouped.get_group('notsorecent'))) / len(diary_df)) * 15)
    old_num = round((int(len(grouped.get_group('old'))) / len(diary_df)) * 15)

    # Choose 10 random movies from the filtered DataFrame
    recommended_movies = pd.concat([high_rated_movies1.sample(n=recent_num), high_rated_movies2.sample(n=notsorecent_num), high_rated_movies3.sample(n=old_num)], ignore_index=True)
    
    return recommended_movies

# Call the recommendation function
recommended_movies = recommend_movies(search_results_df1)
recommended_movies["movie review"]=recommended_movies.apply(lambda row: movie_reception(get_movie_reviews,row['movie title'],row['Release Year']),axis=1)
recommended_movies["review link"]=recommended_movies.apply(lambda row: get_rotten_tomatoes_url(row['movie title'],row['Release Year']),axis=1)

# add people_watched and languages
def people_watched(url):
    r = f"{url}members/"
    r = requests.get(r)
    soup = BeautifulSoup(r.content, "html.parser")
    soup_str = str(soup.find('a', class_="tooltip"))
    start_chr = soup_str.find('title="') + 7
    end_chr = soup_str.find('people">')-1
    people_watched = soup_str[start_chr:end_chr]
    return people_watched

recommended_movies['people_watched'] = recommended_movies['movie_url'].apply(lambda x:people_watched(x))

def find_language(url1):
    languages=[]
    url1=url1+"/details/"
    response = requests.get(url1)
    movie_page=response.text
    soup = BeautifulSoup(movie_page, 'html.parser')
    TAGS=soup.find_all('a')
    for tag in TAGS:
        if "language" in str(tag):
            tag=tag.text
            languages.append(tag)
    return set(languages)

recommended_movies['Language'] = recommended_movies['movie_url'].apply(find_language)
recommended_movies1 = recommended_movies.copy()
recommended_movies1 = recommended_movies1.drop('movie_url', axis=1)

print(recommended_movies1)

Enter user name:  omamori


C:\Users\ADMIN\AppData\Local\Temp\ipykernel_14720\2422878691.py:378: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = diary_df.groupby('Release Year')


                              movie title  Release Year  Rating  \
0                               Midsommar          2019    3.78   
1                             Jojo Rabbit          2019    4.03   
2              Killers of the Flower Moon          2023    4.09   
3       Everything Everywhere All at Once          2022    4.30   
4                        The Green Knight          2021    3.69   
5   Eternal Sunshine of the Spotless Mind          2004    4.25   
6                    Inglourious Basterds          2009    4.34   
7                       Pride & Prejudice          2005    4.10   
8              Captain America: Civil War          2016    3.58   
9                          BlacKkKlansman          2018    3.94   
10                                    Big          1988    3.59   
11                            The Pianist          2002    4.37   
12                              True Lies          1994    3.58   
13                All the President's Men          1976    4.1